In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
import cv2
import os
import matplotlib.pyplot as plt

# Setup configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Adjust path as necessary
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # Adjust based on your dataset
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Adjust threshold as necessary
predictor = DefaultPredictor(cfg)

In [ ]:
fpn_features = {}

def extract_fpn_outputs(module, input, output):
    for name, feature in output.items():
        fpn_features[name] = feature

In [2]:
rpn_outputs = {}

def capture_rpn_outputs(module, input, output):
    # Capture the outputs from the RPN layer; adapt this based on the actual structure observed
    # 'output' is expected to be a tuple where the first item is a list of Instances
    instances = output[0][0]  # Accessing the first Instances object from the list in the first item of the tuple
    
    # Store the relevant outputs for later use; ensure to detach and move to CPU
    rpn_outputs['proposal_boxes'] = instances.proposal_boxes.tensor.detach().cpu()
    rpn_outputs['objectness_logits'] = instances.objectness_logits.detach().cpu()

# Register the hook to the RPN layer; adapt 'proposal_generator' based on your model's structure
handle = predictor.model.proposal_generator.register_forward_hook(capture_rpn_outputs)

In [ ]:
# Register the hook
hook_handle = predictor.model.backbone.register_forward_hook(extract_fpn_outputs)


In [4]:
# Load an image
image_path = 'midtest.jpg'  # Adjust path as necessary
im = cv2.imread(image_path)

# Perform inference (this will trigger the hook)
outputs = predictor(im)

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Assuming 'im' is your original image and 'top_proposal_boxes' contains the top N proposal boxes
im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(im_rgb)

# Iterate over the proposal boxes to draw them
for box in top_proposal_boxes:
    x1, y1, x2, y2 = box
    plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, edgecolor='r', facecolor='none', linewidth=2))

plt.axis('off')
plt.show()

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy.special import softmax

# Assuming 'im' is the original image and has been loaded already
im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) if im.shape[2] == 3 else im

# Normalize the objectness logits to get a probability-like score between 0 and 1
#norm = Normalize(vmin=rpn_outputs['objectness_logits'].min(), vmax=rpn_outputs['objectness_logits'].max())
normalized_logits = softmax(rpn_outputs['objectness_logits'], axis=1)[:, 1]

# Create a blank heatmap with zeros and the same height and width as the original image
heatmap = np.zeros((im.shape[0], im.shape[1]))

# Assuming the proposals are in (x1, y1, x2, y2) format
for box, score in zip(rpn_outputs['proposal_boxes'], normalized_logits):
    x1, y1, x2, y2 = map(int, box)
    # Fill the area within each box on the heatmap with the objectness score
    heatmap[y1:y2, x1:x2] = np.maximum(heatmap[y1:y2, x1:x2], score)

# Compute the scaling factors for width and height
original_height, original_width = im.shape[:2]
scale_x = original_width / heatmap.shape[1]
scale_y = original_height / heatmap.shape[0]

# Resize the heatmap to match the original image size
resized_heatmap = cv2.resize(heatmap, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

# Apply a colormap to the resized heatmap
heatmap_color = cv2.applyColorMap(np.uint8(255 * resized_heatmap), cv2.COLORMAP_JET)

# Overlay the colored heatmap onto the original image
overlayed_img = cv2.addWeighted(im_rgb, 0.5, heatmap_color, 0.5, 0)

# Display the original image and the one with the overlayed heatmap
fig, ax = plt.subplots(1, 2, figsize=(15, 10))
ax[0].imshow(im_rgb)
ax[0].set_title('Original Image')
ax[0].axis('off')

ax[1].imshow(overlayed_img)
ax[1].set_title('RPN Objectness Heatmap Overlay')
ax[1].axis('off')

plt.show()

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
import numpy as np
import cv2
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt

# Assuming 'im' is the original image and 'rpn_outputs' contains the RPN outputs
# Replace 'im' and 'rpn_outputs' with the actual image and RPN output variables

# Normalize the objectness logits to get a probability-like score between 0 and 1
norm = Normalize(vmin=rpn_outputs['objectness_logits'].min(), vmax=rpn_outputs['objectness_logits'].max())
normalized_logits = norm(rpn_outputs['objectness_logits'])

# Create a blank heatmap with zeros and the same height and width as the original image
heatmap = np.zeros((im.shape[0], im.shape[1]), dtype=np.float32)

# Assuming the proposals are in (x1, y1, x2, y2) format
for box, score in zip(rpn_outputs['proposal_boxes'], normalized_logits):
    x1, y1, x2, y2 = map(int, box)
    # Fill the area within each box on the heatmap with the objectness score
    heatmap[y1:y2, x1:x2] = np.maximum(heatmap[y1:y2, x1:x2], score)

# Apply a colormap to the heatmap
heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

# Save the heatmap
heatmap_path = 'rpn_objectness_heatmap.png'
cv2.imwrite(heatmap_path, heatmap_color)

# Display the heatmap
plt.figure(figsize=(10, 10))
plt.imshow(heatmap_color)
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(30, 10))  # Create a 2x3 grid

# Assuming fpn_features contains 5 feature maps
# Flatten the axes array for easier indexing
axes_flat = axes.flatten()

# Iterate through each feature map and its corresponding axis
for i, (level, feature) in enumerate(fpn_features.items()):
    # Plot feature map on the ith subplot
    feature_map = feature[0, 0].detach().cpu().numpy()
    ax = axes_flat[i]
    ax.imshow(feature_map, cmap='viridis')
    ax.set_title(level)
    ax.axis('off')

# If there are less feature maps than subplots, hide the unused subplots
for j in range(i + 1, len(axes_flat)):
    axes_flat[j].set_visible(False)

plt.tight_layout()
plt.savefig('fpn_feature_maps_2x3_layout.png', dpi=600)  # Saving the figure
plt.show()

In [ ]:
handle.remove()

In [ ]:
register_coco_instances("my_dataset_test", {}, "validation.json", "valid")

In [ ]:
from detectron2.data import MetadataCatalog

metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

In [ ]:

outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(im[:, :, ::-1],
               metadata=metadata, 
               scale=0.5, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite('output_filename.jpg', out.get_image()[:, :, ::-1])

In [1]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset  
from detectron2.data.datasets import register_coco_instances   
from detectron2.engine import  DefaultPredictor
from detectron2 import model_zoo
from detectron2.config import get_cfg 
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import cv2
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools 

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
register_coco_instances("my_dataset_test", {}, "test.json", "test")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(predictor.model, val_loader, evaluator) 

my_dataset_test_metadata = MetadataCatalog.get("my_dataset_test")
dataset_dicts = DatasetCatalog.get("my_dataset_test")
   
im = cv2.imread("midtest.jpg")
outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(im[:, :, ::-1],
               metadata=my_dataset_test_metadata, 
               scale=0.5, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imwrite('predmidtest.jpg', out.get_image()[:, :, ::-1])

COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

/home/emon/.local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.804
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.990
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.988
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.808
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.822
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.846
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.846
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.846
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.839
 Average Recall     (AR) @[ IoU=0.50:0.

True

In [ ]:
# Initialize the visualizer with the original image
v = Visualizer(im[:, :, ::-1], metadata=my_dataset_test_metadata, scale=0.5)

# Draw only the bounding boxes on the image
for box in outputs["instances"].pred_boxes:
    v.draw_box(box, edge_color="g").to("cpu")  # Draw each box in green (or any color of choice)

# Save the image with bounding boxes
cv2.imwrite('predmidtest_boxes.jpg', v.get_output().get_image()[:, :, ::-1])

In [ ]:
# Re-initialize the visualizer for mask visualization
v = Visualizer(im[:, :, ::-1], metadata=my_dataset_test_metadata, scale=0.5)

# Draw only the masks on the image
# `pred_masks` is a tensor of shape (N, H, W), where N is the number of detections
for mask in outputs["instances"].pred_masks:
    # Convert tensor to numpy array and ensure it's boolean for `draw_binary_mask`
    mask_array = mask.cpu().numpy()
    v.draw_binary_mask(mask_array.astype(bool), color="g")  # Draw each mask in green (or any color)

# Save the image with masks
cv2.imwrite('predmidtest_masks.jpg', v.get_output().get_image()[:, :, ::-1])


In [ ]:
v = Visualizer(im[:, :, ::-1], metadata=my_dataset_test_metadata, scale=0.5)

# Ensure to move the mask tensors to the CPU before converting to numpy arrays
masks = outputs["instances"].pred_masks.cpu()

# Iterate through each mask and draw it on the image
for mask in masks:
    # Convert the tensor to a numpy array and ensure it's a boolean array
    mask_array = mask.numpy().astype(bool)
    v.draw_binary_mask(mask_array, color="g")  # Change "g" to any desired color

# Save the image with masks
cv2.imwrite('predmidtest_masks.jpg', v.get_output().get_image()[:, :, ::-1])

In [ ]:
v = Visualizer(im[:, :, ::-1], metadata=my_dataset_test_metadata, scale=0.5, instance_mode=ColorMode.IMAGE)

# Extract the predicted boxes and draw them
pred_boxes = outputs["instances"].pred_boxes.tensor.cpu()

# Iterate over each box and draw it
for box in pred_boxes:
    # The `draw_box` method expects a single box in (x1, y1, x2, y2) format, so no conversion is needed here
    v.draw_box(box, edge_color="g")  # You can change "g" to any desired color

# Convert the visualized image back to BGR so it can be saved with OpenCV
visualized_image = v.get_output().get_image()[:, :, ::-1]

# Save the visualized image with bounding boxes
cv2.imwrite('predmidtest_boxes.jpg', visualized_image)

In [ ]:
import torch
import numpy as np
import cv2
from torchvision.transforms.functional import resize

def visualize_objectness_logits(image, objectness_logits, feature_level):
    """
    Visualize objectness logits on the image for a specific feature level (e.g., P2 to P6).
    Args:
    - image: Original image in numpy array format (HxWxC).
    - objectness_logits: Objectness logits tensor (1xCxHxW).
    - feature_level: The pyramid level (e.g., 'P2').
    """
    # Apply sigmoid to convert logits into probabilities
    objectness_probs = torch.sigmoid(objectness_logits).cpu().numpy()

    # Select the objectness probability map for 1:1 aspect ratio anchor (assuming it's the first channel)
    objectness_map = objectness_probs[0, 0, :, :]

    # Resize objectness map to match the original image size
    objectness_map_resized = cv2.resize(objectness_map, (image.shape[1], image.shape[0]))

    # Normalize the objectness map for better visualization
    objectness_map_normalized = cv2.normalize(objectness_map_resized, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Convert to a colored heatmap for visualization
    heatmap = cv2.applyColorMap(objectness_map_normalized, cv2.COLORMAP_JET)

    # Overlay heatmap on the original image
    overlayed_image = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)

    return overlayed_image

In [ ]:
rpn_outputs = {
    "objectness_logits": None,
    "anchor_deltas": None,
}

def capture_rpn_outputs(module, input, output):
    # Assuming the structure of the 'output' dict and capturing the required information
    rpn_outputs["objectness_logits"] = output["pred_objectness_logits"].detach()
    rpn_outputs["anchor_deltas"] = output["pred_anchor_deltas"].detach()

In [ ]:
handle = predictor.model.proposal_generator.register_forward_hook(capture_rpn_outputs)

In [ ]:
feature_level = 'P2'
overlayed_image = visualize_objectness_logits(im, objectness_logits_for_P2, feature_level)
cv2.imwrite(f'overlayed_objectness_{feature_level}.jpg', overlayed_image)
